## Добываем данные

In [1]:
!pip3 install tensorflow-gpu==1.14.0 keras keras-bert keras-rectified-adam bert-tensorflow tqdm wget jupyter numpy pandas

In [31]:
import wget
import os
from zipfile import ZipFile
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from keras_radam import RAdam
from keras.models import Model, load_model
import warnings
warnings.simplefilter("ignore", category=PendingDeprecationWarning)
from keras_bert import load_trained_model_from_checkpoint, get_custom_objects, Tokenizer
import codecs
from bert.tokenization import FullTokenizer
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
from keras import backend as K


SEQ_LEN = 512
LR = 1e-5

In [3]:
dataset_creator_url = 'https://github.com/rkadlec/ubuntu-ranking-dataset-creator/archive/master.zip'
creator_filename = wget.download(dataset_creator_url)
with ZipFile(creator_filename, 'r') as zipfile:
    zipfile.extractall()

In [4]:
os.chdir('./ubuntu-ranking-dataset-creator-master/src/')

In [5]:
! chmod 777 generate.sh
! pip2 install --user --upgrade pip -r ../requirements.txt

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Requirement already up-to-date: pip in /home/lexi/.local/lib/python2.7/site-packages (19.3.1)
Requirement already up-to-date: six>=1.10.0 in /home/lexi/.local/lib/python2.7/site-packages (from -r ../requirements.txt (line 1)) (1.12.0)
Requirement already up-to-date: unicodecsv>=0.14.0 in /home/lexi/.local/lib/python2.7/site-packages (from -r ../requirements.txt (line 2)) (0.14.1)
Requirement already up-to-date: nltk>=3.1 in /home/lexi/.local/lib/python2.7/site-packages (from -r ../requirements.txt (line 3)) (3.4.5)


In [6]:
# change python commands in generate.sh to python2
! ./generate.sh

[nltk_data] Downloading package punkt to /home/lexi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lexi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Successfully downloaded ./ubuntu_dialogs.tgz
Unpacking dialogs ...
Archive unpacked.
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
1

In [7]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('valid.csv')

In [8]:
os.chdir('../../')

## Анализ данных

In [9]:
train_data.columns, test_data.columns

(Index(['Context', 'Utterance', 'Label'], dtype='object'),
 Index(['Context', 'Ground Truth Utterance', 'Distractor_0', 'Distractor_1',
        'Distractor_2', 'Distractor_3', 'Distractor_4', 'Distractor_5',
        'Distractor_6', 'Distractor_7', 'Distractor_8'],
       dtype='object'))

In [10]:
train_data.tail()

,Context,Utterance,Label
999995,'booby raid' is __eou__ hehe .. im jsut tring to get silicon raid image a /dev/something so i can cp my filesystem to it and edit fstab __eou__ mod kernel get /dev/something... then it should be all set. __eou__ damm.. so no one has done anything with raid as the primary file system in here? __eou__ __eot__ I installed onto a software raid1 array once __eou__ __eot__ how u manage that i tried a software raid1 it wouldnt work during install. __eou__ __eot__,I think I made a plain /boot and then put a partition on each disk to be the raid'd / __eou__,1.0
999996,"does gcc come installed by default on ubuntu5.10? __eou__ __eot__ no >> sudo apt-get install build-essential <<will sort that out for you __eou__ __eot__ dailup, was hopeing it at least had the libs __eou__ __eot__ it is on the CD afik __eou__ it is on the CD I just checked __eou__ __eot__",how do I tweak the xorg.conf fire to use GLX and OpenGL __eou__,0.0
999997,"anyone here have kubuntu-desktop installed __eou__ __eot__ I tried it, but it sucked, so I removed it again. #kubuntu might be a good place to ask, too __eou__ __eot__ they're biased __eou__ __eot__","so are we :) __eou__ kubuntu isn't as complete as ubuntu, in my experience. they don't have anything to tell them when to update their system, for instance __eou__",1.0
999998,"well __eou__ i wish someone would talk about my system recovery actually... __eou__ anybody has any idea about what it means when my file system is mounted read-only after a system crash... __eou__ __eot__ have you tried just rebooting it again after that? __eou__ __eot__ : but anyway my options are rather limited since I cant actually edit anything being in read-only mode... __eou__ __eot__ true. You could try to remount. Typically, it just means that it's fixed it, but wants you to reboot anyway. __eou__ __eot__",sure not. I remember a live windows but I can't remember the name. SAM part? __eou__,0.0
999999,"i'm having a problem with hedgehog, I am trying to use dialup. The modem initalises, and dials the number... but firefox won't connect. I can't even ping out. Any ideas? __eou__ still connected __eou__ __eot__ Perhaps you need to set ppp0 as default gateway from System->Administration->Networking __eou__ __eot__ it is already set as default __eou__ __eot__ Is it only problem with FF? __eou__ __eot__ no, i cannot even ping out. it's like there's no connection __eou__ __eot__",this thread may help: http://ubuntuforums.org/showthread.php?t=1740368 __eou__,0.0


In [11]:
test_data.head()

,Context,Ground Truth Utterance,Distractor_0,Distractor_1,Distractor_2,Distractor_3,Distractor_4,Distractor_5,Distractor_6,Distractor_7,Distractor_8
0,"anyone knows why my stock oneiric exports env var 'USERNAME'? I mean what is that used for? I know of $USER but not $USERNAME . My precise install doesn't export USERNAME __eou__ __eot__ looks like it used to be exported by lightdm, but the line had the comment ""// FIXME: Is this required?"" so I guess it isn't surprising it is gone __eou__ __eot__ thanks! How the heck did you figure that out? __eou__ __eot__ https://bugs.launchpad.net/lightdm/+bug/864109/comments/3 __eou__ __eot__",nice thanks! __eou__,"wrong channel for it, but check efnet.org, unofficial page. __eou__","every time the kernel changes, you will lose video __eou__ yep __eou__",ok __eou__,!nomodeset > acer __eou__ I'm assuming it is a driver issue. __eou__ !pm > acer __eou__ i DON'T pm. ;) __eou__ OOPS SORRY FOR THE CAPS __eou__,"http://www.ubuntu.com/project/about-ubuntu/derivatives (some call them derivatives, others call them flavors, same difference) __eou__",thx __eou__ unfortunately the program isn't installed from the repositories __eou__,how can I check? By doing a recovery for testing? __eou__,my humble apologies __eou__,#ubuntu-offtopic __eou__
1,"i set up my hd such that i have to type a passphrase to access it at boot. how can i remove that passwrd, and just boot up normal. i did this at install, it works fine, just tired of having reboots where i need to be at terminal to type passwd in. help? __eou__ __eot__ backup your data, and re-install without encryption ""might"" be the easiest method __eou__ __eot__","so you dont know, ok, anyone else? __eou__ you are like, yah my mouse doesnt work, reinstall your os lolol what a joke __eou__","nmap is nice, but it wasn't what I was looking for. I finally found it again: mtr (my traceroute) is what I was looking for. I'll be keeping nmap handy though. __eou__",ok __eou__,"cdrom worked fine on windows. __eou__ i dont think it has anything to do with the buring process, cds work fine on my desktop and my other ubuntu lap __eou__","ah yes, i have read return as rerun __eou__",hm? __eou__,"not the case, LTS is every other .04 release. The .04 isn't always more stable __eou__ I would reinstall with Precise __eou__ you can restore user data and such from backup __eou__",Pretty much __eou__,I used the one I downloaded from AMD __eou__,"ffmpeg is part of the package , quixotedon , at least I'm quite sure it still is __eou__ if not just install ffmpeg __eou__"
2,im trying to use ubuntu on my macbook pro retina __eou__ i read in the forums that ubuntu has a apple version now? __eou__ __eot__ not that ive ever heard of.. normal ubutnu should work on an intel based mac. there is the PPC version also. __eou__ you want total control? or what are you wanting exactly? __eou__ __eot__,just wondering how it runs __eou__,"yes, that's what I did, exported it to a ""id_dsa"" file, then back to Ubuntu copied it into ~/.ssh/ __eou__",nothing - i am talking about the question of myhero __eou__,that should fix the fonts being too large __eou__,"okay, so hcitool echos back hci0 <mac address of controller> but the bluetooth devices panel keeps disconnecting and reconnecting the device (or so it seems) any idea why that would be? __eou__","I get to the menu with options such as 'try ubuntu', 'install ubuntu', 'check disc' __eou__","why do u need analyzer __eou__ it is a toy __eou__ ok msp301 __eou__ but y, i mean it is the same ubunut, only with older programs __eou__ ubuntu 804 or 1204 __eou__ no i dont use 804 __eou__ i am asking hypo qs __eou__",Cntrl-C may stop the command but it doesn't fix my HDD problem. __eou__,"if you're only going to run Ubuntu, just get a normal PC rather than a mac __eou__ that said, I'm running it on a macbook, because I got one relatively cheaply __eou__",the ones which are not picked up at the moment are on STDERR and not STDOUT and > is only covering S

Наши данные состоят только из контекста, реплики и метки, является ли эта реплика продолжением контекста или нет. Её можно вопринимать как целевую переменную. Поэтому EDA, например, распределение одних фичей относительно других и их возможный вклад в целевую переменную здесь не имеет смысла. Целевая переменная бинарная, по ней тоже ничего особенного посмотришь. Вообще датасет довольно straightforward, т.е. понятно, для чего конкретно он предназначен.

Наши данные - примеры диалогов с довольно длинным контекстом перед целевой репликой, что позволяет учитывать больший контекст в задаче question answering (и в принципе в задаче генерации близких к естественным реплик чат-бота), тогда как в классической постановке задачи существуют только пары "предыдущее предложение - ответная реплик", а контекст учитывается с помощью разных dialogue state tracker.

Теоретически, с помощью подобного корпуса можно было бы решать следующие задачи:
- генеративная задача предсказания следующей реплики (Next Sentence Prediction)
- дискриминативная задача выбора лучшей реплики из нескольких (на что намекает строение test data, содержащего 10 вариантов ответа для каждого контекста)

Другие NLP-задачи можно обучать на данном корпусе только с некоторой доразметкой. Например, если каждому Context сопоставить тему с помощью ручной разметки, то можно будет обучать Topic Modeling, что позволит Убунту-форуму отправлять чаты в разделы соответстующей тематики.

Однако практически я считаю, что этот корпус не позволит обучить достаточно хорошую модель генерации ответных реплик, потому что:
- ответы в этом Убунту-корпусе довольно технические, требуют external knowledge
- последняя реплика контекста - далеко не всегда вопрос, и ответные реплики могут быть непредсказуемыми, например, содержать новый вопрос с уточнением

При этом задача выбора наиболее подходящего ответа из нескольких выглядит многообещающе. В реальной ситуации может быть такое, что чат-бот с помощью какой-то другой модели и/или набора шаблонов генерирует варианты ответов, а модель, обученная на Убунту-корпусе, выбирает из них лучший. Точнее, это был бы бейзлайн, потому что в данном корпусе в качестве неправильных ответов (distractor) предлагаются рандомные реплики, а для улучшения качества надо бы в качестве дистракторов предлагать реплики, к примеру, одной тематики.

В следующем разделе я реализую задачу выбора лучшей реплики из нескольких с помощью предоставленного датасета, условно назвав её Next Sentence Choice.

## Next Sentence Choice

Самый важный шаг в NLP - перевод текста в эмбеддинги, и не секрет, что с недавних пор лучшими являются BERT embeddings. По счастливому совпадению, BERT обучался на двух задачах - Masked Language Modeling и Next Sentence Prediction, и вторая, с небольшими изменениями, именно то, что нам надо.

Поэтому бейзлайн - написать препроцессинг наших данных, загнать его в Берт и померить качество.

In [12]:
# Скачать подходящую бертовскую модель
bert_model_name='google-bert'
bert_model_dir = './models/'
bert_model_path = os.path.join(bert_model_dir, bert_model_name)

if not os.path.exists(bert_model_path):
    if not os.path.exists(bert_model_dir):
        os.mkdir(bert_model_dir)
    bert_url = 'https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip'
    wget.download(bert_url, out=bert_model_path+'.zip')
    zipdata = ZipFile(bert_model_path+'.zip')
    zipinfos = zipdata.infolist()
    for zipinfo in zipinfos:
        zipinfo.filename = zipinfo.filename.replace('uncased_L-12_H-768_A-12', bert_model_name)
        zipdata.extract(zipinfo, path=bert_model_dir)
    print("Unzipped")

Unzipped


In [13]:
# Загрузить её в keras с помощью библиотеки keras-bert
config_path = os.path.join(bert_model_path, 'bert_config.json')
checkpoint_path = os.path.join(bert_model_path, 'bert_model.ckpt')
vocab_path = os.path.join(bert_model_path, 'vocab.txt')

model = load_trained_model_from_checkpoint(
        config_path,
        checkpoint_path,
        training=True,
        trainable=True,
)

In [14]:
# Архитектура модели
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [15]:
model.output

[<tf.Tensor 'MLM/Identity:0' shape=(?, 512, 30522) dtype=float32>,
 <tf.Tensor 'NSP/Softmax:0' shape=(?, 2) dtype=float32>]

In [16]:
# Из двух выходов модели нам нужен NSP - Next Sentence Prediction
inputs = model.inputs[:2]
outputs = model.get_layer('NSP').output
new_model = Model(inputs, outputs)

In [17]:
# Способ разметки авторов датасета достаточно странный, пытаемся восстановить оригинальный текст.
# Эта функция костыльная, но в первом приближении её хватает.
# Конец реплики заменяем на точку, переход речи к другому персонажу - на перевод строки.
def remove_eou_eot(sent):
    return sent.replace(' __eou__', '.').replace(' __eot__', '\n')

In [18]:
tokenizer = FullTokenizer(vocab_file='models/google-bert/vocab.txt', do_lower_case=True)

In [19]:
# Предобученная бертовская модель работает только с предложениями не больше 512 WordPiece, так что увы, обрезаем.
# Попробуем обрезать спереди. Это антиинтуитивно, зато реплика непосредственно перед ответом сохранится полностью.
def truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop(0)
        else:
            tokens_b.pop(0)

In [20]:
# Превращаем в бертовский формат: контекст и предполагаемый ответ слепляем в одно предложение через SEP
# и кодируем индексами из бертовского словаря
def make_pairs(sent_a, sent_b, tokenizer, seq_len=512):
    tokenized_a = tokenizer.tokenize(remove_eou_eot(sent_a))
    tokenized_b = tokenizer.tokenize(remove_eou_eot(sent_b))
    truncate_seq_pair(tokenized_a, tokenized_b, seq_len - 3)
    segments = [0] * (len(tokenized_a) + 2) + [1] * (len(tokenized_b) + 1)
    tokens = ['[CLS]'] + tokenized_a + ['[SEP]'] + tokenized_b + ['[SEP]']
    ids = tokenizer.convert_tokens_to_ids(tokens)
    while len(ids) < seq_len:
        ids.append(0)
        segments.append(0)
    return [ids, segments]

In [21]:
np.array(make_pairs(val_data.iloc[0]['Context'], val_data.iloc[0]['Ground Truth Utterance'], tokenizer))

array([[ 101, 2151, 4784, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]])

In [22]:
# Создаём тест-сет: по десять пар, из которых первая - истинная, другие девять - ложные
def make_test_set(tokenizer, test_len=1000):
    ids, segments = list(), list()
    for i in tqdm(range(test_len)):
        tokenized = make_pairs(test_data.iloc[i]['Context'], test_data.iloc[i]['Ground Truth Utterance'], tokenizer)
        ids += [tokenized[0]]
        segments += [tokenized[1]]
        for j in range(9):
            tokenized_tmp = make_pairs(test_data.iloc[i]['Context'], test_data.iloc[i]['Distractor_'+str(j)], tokenizer)
            ids += [tokenized_tmp[0]]
            segments += [tokenized_tmp[1]]
    return np.array([ids, segments])

In [23]:
test_set = make_test_set(tokenizer)

In [24]:
%%time
predicted = new_model.predict([test_set[0], test_set[1]])


CPU times: user 33.2 s, sys: 9.94 s, total: 43.1 s
Wall time: 6min 4s


In [25]:
# Измеряем accuracy: поскольку ground truth по построению всегда нулевой (по индексу), считаем долю нулей.
def acc(predicted):
    answers = []
    for i in range(10, len(predicted) + 1, 10):
        answers.append(np.argmax(predicted[:i, 0]) % 10)
    return np.count_nonzero(np.array(answers) == 0) * 10 / len(predicted)

In [26]:
acc(predicted)

0.997

Как можно видеть, точность с pretrained bert на данном датасете довольно высока. В реальных задачах всё-таки хорошо бы выбирать лучший не из рандомных ответов, а более-менее подходящих, например, сгенерированных по шаблонам или полученных с помощью модели, обученной на каких-то других данных.

Но вообще модель можно дообучить, fine-tune, для берта для разных задач хватает менее 5 эпох. Я не успела его дописать, но выглядит это примерно так:

## Training

In [27]:
# Делаем train set. Он состоит из пар "контекст - истинное продолжение", "контекст - ложное продолжение"
def make_train_set(tokenizer, train_len=100):
    ids, segments, y = list(), list(), list()
    for i in tqdm(range(train_len)):
        tokenized = make_pairs(train_data.iloc[i]['Context'], train_data.iloc[i]['Utterance'], tokenizer)
        ids += [tokenized[0]]
        segments += [tokenized[1]]
        y += [[int(train_data.iloc[i]['Label']), (int(train_data.iloc[i]['Label'] + 1) % 2)]]
    return np.array([ids, segments]), np.array(y)

In [28]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf'''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [29]:
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [32]:
rms = RAdam(lr=LR)
new_model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])

In [ ]:
x, y = make_train_set(tokenizer)
new_model.fit([x[0], x[1]], y, batch_size=1, epochs=1, verbose=20)